## Dependencies

In [22]:
import sagemaker
import boto3
from datetime import datetime
from io import StringIO
import pandas as pd
import numpy as np

from pyathena import connect

In [24]:
sess = sagemaker.Session() #Intialzie session

bucket = sess.default_bucket()
RD_Bucket = 'policedatasetbucket' #Raw Data
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
s3 = boto3.client('s3') # Create an S3 client
s3_resource = boto3.resource('s3') # Create an S3 resource

print('Bucket - > ',bucket)
print('Role - > ',role)
print('Region - > ',region)
print('S3 - > ',s3)
print('S3 Resource - > ',s3_resource)

DataNotFoundError: Unable to load data for: sagemaker

In [4]:
# Function to verify if bucket exist, if not create
def verify_create_bucket(bucket_name):
    response = s3.list_buckets()
    for bucket in response['Buckets']:
        if bucket['Name'] == bucket_name:
            print(f"The {bucket_name} bucket exists.")
            break
    else:
        print(f"The {bucket_name} bucket does not exist, creating")
        !aws s3 mb s3://{RD_Bucket}/

In [5]:
verify_create_bucket(RD_Bucket)

The policedatasetbucket bucket exists.


In [ ]:
#function to download yearly data
def Get_Data(year):
    url = f"https://seshat.datasd.org/pd/pd_calls_for_service_{year}_datasd.csv"
    df = pd.read_csv(url)
    return df

In [ ]:
# List of years since 2018
Years = list(range(2018,datetime.now().year+1))
Years

In [ ]:
#iterate over years
for year in Years:
    year_df = Get_Data(year)
    csv_buffer = StringIO()
    year_df.to_csv(csv_buffer, index=False)
    file = 'SDPD_Calls_' + str(year) + '.csv'
    s3_resource.Object(RD_Bucket,file).put(Body=csv_buffer.getvalue())
    print(file,'loaded in',RD_Bucket,'bucket')

In [ ]:
def URL_2_Bucket(url,file_name):
    df = pd.read_csv(url, low_memory=False)
    csv_buffer = StringIO()
    df.to_csv(csv_buffer, index=False)
    file = file_name + '.csv'
    s3_resource.Object(RD_Bucket,file).put(Body=csv_buffer.getvalue())
    print(file,'loaded in',RD_Bucket,'bucket')
    

In [ ]:
type_url = f"http://seshat.datasd.org/pd/pd_cfs_calltypes_datasd.csv"
dipo_url = f"http://seshat.datasd.org/pd/pd_dispo_codes_datasd.csv"
ripa_stops_url = "https://seshat.datasd.org/pd/ripa_stops_datasd.csv"
ripa_stops_dic = "https://seshat.datasd.org/pd/ripa_stops_dictionary_datasd.csv"

In [ ]:
URL_2_Bucket(type_url,'Type')
URL_2_Bucket(dipo_url,'Dispo')
URL_2_Bucket(ripa_stops_url,'Ripa_Stops')
URL_2_Bucket(ripa_stops_dic,'Ripa_Stops_Dic')

# Create Athena Database 

In [15]:
ingest_create_athena_db_passed = False

In [12]:
database_name = "sdpolicedb"

In [17]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [18]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

DataNotFoundError: Unable to load data for: sdk-default-configuration